In [142]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import datetime as dt

print('Imports completed!')

Imports completed!


In [143]:
# Read in game score data
game_df = pd.read_excel('./data/apiGameScores.xlsx')
game_df

,game_ids,date,visitor,home,visitor_points,home_points
0,319,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,1300,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,1344,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,1353,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,1376,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...,...
79,12202,2023-03-31T23:30:00.000Z,Atlanta Hawks,Brooklyn Nets,107.0,124.0
80,13399,2024-03-01T00:30:00.000Z,Atlanta Hawks,Brooklyn Nets,97.0,124.0
81,13414,2024-03-02T20:00:00.000Z,Atlanta Hawks,Brooklyn Nets,102.0,114.0
82,15092,2025-03-16T22:00:00.000Z,Atlanta Hawks,Brooklyn Nets,NaN,NaN


In [144]:
# Grab game ids to be used with api call
game_ids = game_df['game_ids'].values
game_ids

array([  319,  1300,  1344,  1353,  1376,  2125,  2709,  3065,  3149,
        4654,  5069,  7134,  8671,  9484,  9780, 10302, 11263, 12388,
       12395, 12404, 13582, 13608, 13877, 14211,   238,   498,  1362,
        1370,  1380,  2417,  3686,  4121,  4806,  5420,  6906,  7160,
        7959,  8619, 10421, 10492, 12265, 12375, 12381, 12399, 12786,
       13277, 14271, 14701,   171,   712,  2481,  2619,  3261,  3540,
        4237,  5373,  6698,  7267,  7782,  9170, 10728, 11566, 11960,
       12757, 13757, 14118,   268,  2101,  2680,  2957,  3247,  4822,
        4995,  6820,  6975,  8238,  8253,  8899, 11429, 12202, 13399,
       13414, 15092, 15283], dtype=int64)

In [145]:
url = "https://api-nba-v1.p.rapidapi.com/games/statistics"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

statistics = []
for game_id in game_ids:
	querystring = {"id": game_id}
	response = requests.get(url, headers=headers, params=querystring)
	
 	# Check if the API request was successful
	if response.status_code == 200:
		game_statistics = response.json()
  
		# Extract game id
		id = game_statistics['parameters']['id']

		# Extract visitor and home team names
		visitor = game_statistics['response'][0]['team']['name']
		home = game_statistics['response'][1]['team']['name']

		# Extract statistics for visitor team
		visitor_statics = game_statistics['response'][0]['statistics'][0]
		visitor_stats = {f"visitor_{key}":value for key, value in visitor_statics.items()}
		visitor_stats['game_id'] = id
		visitor_stats['visitor_team'] = visitor
		
		# Extract statistics for home team
		home_statics = game_statistics['response'][1]['statistics'][0]
		home_stats = {f"home_{key}":value for key, value in home_statics.items()}
		home_stats['game_id'] = id
		home_stats['home_team'] = home

		# Append both visitor and home stats
		statistics.append({**visitor_stats, **home_stats})

# Create the Data Frame
game_statistics_df = pd.DataFrame(statistics)
game_statistics_df


,visitor_fastBreakPoints,visitor_pointsInPaint,visitor_biggestLead,visitor_secondChancePoints,visitor_pointsOffTurnovers,visitor_longestRun,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,...,home_defReb,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,home_min,home_team
0,18,34,2,NaN,20,NaN,97,34,82,41.5,...,37,43,33,25,8,20,7,24,240:00,Atlanta Hawks
1,25,46,12,NaN,18,NaN,107,40,90,44.4,...,39,44,31,22,10,17,10,11,240:00,Atlanta Hawks
2,15,36,3,NaN,10,NaN,101,37,102,36.3,...,40,53,23,20,4,11,9,1,240:00,Atlanta Hawks
3,8,36,0,NaN,6,NaN,72,28,88,31.8,...,46,49,20,16,7,15,15,17,240:00,Atlanta Hawks
4,8,32,10,NaN,9,NaN,83,29,77,37.7,...,38,51,30,17,13,12,3,27,240:00,Atlanta Hawks
5,9,28,20,12.0,9,14.0,103,36,83,43.4,...,31,43,22,17,5,12,5,-2,240:00,Atlanta Hawks
6,10,36,3,19.0,22,9.0,116,37,90,41.1,...,39,52,26,30,7,17,4,7,240:00,Atlanta Hawks
7,4,34,10,4.0,7,13.0,110,39,81,48.1,...,31,36,24,22,8,14,4,-3,240:00,Atlanta Hawks
